In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving vehicle.zip to vehicle.zip
User uploaded file "vehicle.zip" with length 150238593 bytes


In [ ]:
from zipfile import ZipFile

In [ ]:
from zipfile import ZipFile
file_name = "/content/vehicle.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import tensorflow as tf
import os
import numpy as np

In [ ]:
base_dir=r"/content/vehicle"

In [ ]:
#preprocessing

In [ ]:
IMAGE_SIZE=224
BATCH_SIZE=64

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(

           rescale=1./255,
           zoom_range=0.2,
           horizontal_flip=True,
           validation_split=0.1)

validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
         rescale=1./255,
         validation_split=0.1,
)

In [ ]:
train_genarator=train_datagen.flow_from_directory(
       base_dir,
       target_size=(IMAGE_SIZE,IMAGE_SIZE),
       batch_size=BATCH_SIZE,
       subset='training'

    )

validation_generator=validation_datagen.flow_from_directory(
       base_dir,
       target_size=(IMAGE_SIZE,IMAGE_SIZE),
       batch_size=BATCH_SIZE,
       subset='validation'

    )

Found 3580 images belonging to 2 classes.
Found 397 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob


In [ ]:
IMAGE_SIZE=[224,224]
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [ ]:
for layer in vgg.layers:
  layer.trainable=False

In [ ]:
folders=glob(r"/content/vehicle/*")
print(len(folders))

2


In [ ]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epoch=10

history=model.fit(train_genarator,
                  steps_per_epoch=len(train_genarator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                     )

Epoch 1/10
56/56 [==============================] - 1547s 28s/step - loss: 0.6548 - accuracy: 0.6159 - val_loss: 0.4645 - val_accuracy: 0.6776
Epoch 2/10
56/56 [==============================] - 1537s 28s/step - loss: 0.6173 - accuracy: 0.6263 - val_loss: 0.3899 - val_accuracy: 0.7809
Epoch 3/10
56/56 [==============================] - 1532s 27s/step - loss: 0.6312 - accuracy: 0.6355 - val_loss: 0.4001 - val_accuracy: 0.7406
Epoch 4/10
50/56 [=========================>....] - ETA: 2:22 - loss: 0.5965 - accuracy: 0.6151

In [ ]:
#prediction

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_pred = image.load_img(r"/content/Ambulance.yolo/63H17H33KXHS.jpg", target_size=(224, 224))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis=0)

# Make predictions
result = model.predict(img_pred)

# Print the prediction
if result[0][0] > result[0][1]:
     prediction = "ambulance"
else:
     prediction = "vehicle"

print(prediction)


1/1 [==============================] - 0s 376ms/step
vehicle
